# Simple RAG (Retrieval-Augmented Generation) System for CSV Files

## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying CSV documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

## Key Components

1. Loading and spliting csv files.
2. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
3. Retriever setup for querying the processed documents
4. Creating a question and answer over the csv data.

## Method Details

### Document Preprocessing

1. The csv is loaded using langchain Csvloader
2. The data is split into chunks.


### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store (in memory vector store, not persistent like pinecone. You can store though but for the sake tutorial, not needed here) is created from these embeddings for efficient similarity search.

### Retriever Setup

1. A retriever is configured to fetch the most relevant chunks for a given query.

## Benefits of this Approach

1. Scalability: Can handle large documents by processing them in chunks.
2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results.
3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
4. Integration with Advanced NLP: Uses OpenAI embeddings for state-of-the-art text representation.

## Conclusion

This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within a csv file.

import libries

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install faiss-cpu langchain langchain-community langchain-openai pandas python-dotenv

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/customers-100.csv https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv


In [ ]:
import pandas as pd

file_path = ('data/customers-100.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


load and process csv data

In [ ]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split() # Loads the CSV and splits it into smaller documents (chunks) to improve retrieval efficiency and manage context for the language model.

Initiate faiss vector store and openai embedding

In [ ]:
import faiss # Import the FAISS library for efficient similarity search
from langchain_community.docstore.in_memory import InMemoryDocstore # Import In-memory document store for FAISS
from langchain_community.vectorstores import FAISS # Import FAISS vector store class

embeddings = OpenAIEmbeddings() # Initialize OpenAI embeddings model
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" "))) # Create a FAISS index for L2 distance similarity, determining vector dimension from an embedding query
vector_store = FAISS( # Initialize the FAISS vector store
    embedding_function=OpenAIEmbeddings(), # Use OpenAI embeddings to generate vectors
    index=index, # Provide the pre-created FAISS index
    docstore=InMemoryDocstore(), # Use an in-memory document store
    index_to_docstore_id={} # Initialize an empty mapping for index to document ID
)

Add the splitted csv data to the vector store

In [ ]:
vector_store.add_documents(documents=docs)

Create the retrieval chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate # Import ChatPromptTemplate to create structured chat prompts
from langchain.chains import create_retrieval_chain # Import create_retrieval_chain to build a RAG chain
from langchain.chains.combine_documents import create_stuff_documents_chain # Import create_stuff_documents_chain to combine retrieved documents

retriever = vector_store.as_retriever() # Convert our vector store into a retriever that can fetch relevant documents

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. " # Define the role of the AI assistant
    "Use the following pieces of retrieved context to answer " # Instruct the AI to use provided context for answers
    "the question. If you don't know the answer, say that you " # Tell the AI to state if it doesn't know the answer
    "don't know. Use three sentences maximum and keep the " # Set a limit of three sentences for conciseness
    "answer concise." # Emphasize keeping the answer brief
    "\n\n" # Add a new line for formatting
    "{context}" # Placeholder for the retrieved context documents
)

prompt = ChatPromptTemplate.from_messages([ # Create a chat prompt template
    ("system", system_prompt), # Assign the system prompt to the 'system' role
    ("human", "{input}"), # Assign the user's question to the 'human' role

])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt) # Combine our language model (llm) with the prompt to answer questions based on documents
rag_chain = create_retrieval_chain(retriever, question_answer_chain) # Create the full RAG chain, linking the retriever to the question-answer chain

Query the rag bot with a question based on the CSV data

In [ ]:
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

'Sheryl Baxter works for Rasmussen Group.'

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--simple-csv-rag)